# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [3]:
# импортирую библиотеку pandas
import pandas as pd

# считываю файл, сохраняя в переменной data
data = pd.read_csv('/datasets/data.csv')

# вывожу первые 10 строк таблицы
display(data.head(10))


# методом info просматриваю информацию о таблице
data.info()

# проверяю названия колонок
print(data.columns)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


**Вывод**



Видно, что мы имеем 21 525 строк и 12 колонок. В days_employed и total_income одинаковое количество значений, но если сравнивать с остальными - разное. Это говорит о том, что в days_employed и total_income встречаются 'нулевые'/пропущенные значения. Проверка именования столбцов показала, что все значения прописаны верно. В датафрэйме встречаются различные типы данных, такие как int64, float64 и object.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# методом isna() в совокупности с sum() проверяю и подсчитываю пропуски
data.isna().sum()

# методом loc заменяю пропущенные значения в days_employed и total_income на медиану.
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].sort_values(ascending=True).median()
data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].sort_values(ascending=True).median()


# методом loc заменяю нулевые знаения возраста на среднее
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].sum() / data['dob_years'].count()


**Вывод**


Пропуски в колонках days_employed и total_income были заполнены медианой, так как среднее(на мой взгляд) в этом случае использовать не корректно. В колонке с возрастом были обнаружены нули - заменила их на среднее значение. Полгаю, что days_employed и total_income не заполнены умышленно, в том плане, что люди не захотели делиться стажем и зп. Странно, что в возрасте были нули, думаю, что это ошибка связанна с человеческим фактором.

### Замена типа данных

In [12]:
# меняю отработку в часах на дни, применив метод loc
data.loc[data['days_employed'] >= 30000, 'days_employed'] = data['days_employed'] / 24

# привожу значения в колонках days_employed и total_income к челочисленным и делаю их положительными.
data['days_employed'] = abs(data['days_employed'].astype('int'))
data['total_income'] = abs(data['total_income'].astype('int'))
data['dob_years'] = data['dob_years'].astype('int')
display(data)





,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**


При просмотре таблицы были замечены странные значения в days_employed и total_income. Данные были переведены в тип int, отрицательные значения изменены на положительные. Так же, странные(большие) значения в days_employed (предположительно записанные в минутах) были переведены в часы. В колонке dob_years данные были в вещественном типе - заменила на целочисленные. Метод astype был применен, так как данный метод удобен в использовании для перевода в нужный тип. Фунцкия abs возвращает абсолютное значение аргумента, абсолютное значение любого числа с отрицательным значением - это всегда соответствующее положительное, поэтому я применила конкретно эту функцию для отрицательных чисел в таблице.

### Обработка дубликатов

Находим и подсчитываем количество дубликатов

In [13]:
# находим количество дубликатов
data.duplicated().sum()


54

Проверяем стобцы на наличие неявных дубликатов. При необходимости заменяем.

In [14]:
# проверяю столбец education на наличие неявных дубликатов
data['education'].unique()
# сделала функцию, которая заменяет неправильные значение на правильные
def correct_educat(wrong_values, correct_value):
    for wrong_value in wrong_values:
        data['education'] = data['education'].replace(wrong_values, correct_value)
dupli1 = ['Среднее', 'СРЕДНЕЕ', 'cреднее']
correct_name1 = 'среднее'
dupli2 = ['ВЫСШЕЕ', 'Высшее']
correct_name2 = 'высшее'
dupli3 = ['НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее']
correct_name3 = 'неоконченное высшее'
dupli4 = ['НАЧАЛЬНОЕ', 'Начальное']
correct_name4 = 'начальное'
dupli5 = ['Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ']
correct_name5 = 'ученая степень'

# применяю функцию, избавляюсь от дублей
correct_educat(dupli1, correct_name1)
correct_educat(dupli2, correct_name2)
correct_educat(dupli3, correct_name3)
correct_educat(dupli4, correct_name4)
correct_educat(dupli5, correct_name5)



# проверяю столбец children на наличие неявных дубликатов
data['children'].sort_values(ascending = True).unique()
# заменяю значение -1 на положительное в колонке children
data['children'] = data['children'].replace(-1, 1)
# смотрю, сколько строк мы имеем где детей = 20
data[data['children'] == 20]
# заменяем опечатку '20' на '2'
data['children'] = data['children'].replace(20, 2)



# проверяю столбец debt на наличие неявных дубликатов
data['debt'].unique()

# проверяю столбец family_status на наличие неявных дубликатов
data['family_status'].unique()

# проверяю столбец puspose на наличие неявных дубликатов
data['purpose'].unique()
# сортирую их в порядке возрастания
data['purpose'].sort_values(ascending = True).unique()

data['income_type'].unique()


array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Удаляем дубликаты и проверяем количество

In [15]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

**Вывод**



Были преобразованы дубли в колонке education, возникшие при заполнении. Хотелось бы отметить, что значение 'среднее' имело латинскую букву 'c' - этот момент был учтен и тоже заменен. Найдены странные значения в колонке children. Значения -1 поменяла на положительную цифру - думаю что знак минус поставлен ошибочно или возник из-за ошибки при выгрузке. 20 детей заменила на 2, думаю, что это опечатка.

### Лемматизация

Подсчитываем количество упоминаний в столбце purpose, импортируя нужные библиотеки и применив лемматизацию

In [16]:
# импортирую библиотеки
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

# нахожу уникальные знаения столбца purpose, лемматизирую и подсчитываю количество упоминаний
text = data['purpose'].unique()
separator = ' '
text = separator.join(text)
lemmas = m.lemmatize(text)
Counter(lemmas)






Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Из получившихся значений определяем "ключевые". Прописав функцию, добавляем новый столбец с лемматизированными значениями методом apply()

In [17]:
# прописываю функцию
def purpose_lemmas(row):
    purpose = row['purpose']
    lemmas_purpose = m.lemmatize(purpose)
    if ('недвижимость' in lemmas_purpose or 'жилье' in lemmas_purpose):
        return 'недвижимость'
    elif 'автомобиль' in lemmas_purpose:
        return 'автомобиль'
    elif 'образование' in lemmas_purpose:
        return 'образование'
    elif 'свадьба' in lemmas_purpose:
        return 'свадьба'
    else:
        return 'цель не определена'

# добавляю новый столбец с корректными(лемматизированными) значениями
data['lemmas_purposes'] = data.apply(purpose_lemmas, axis = 1)
display(data.head(10))



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purposes
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод** 

Была проведена лемматизация столбца purpose. Ключевые цели: недвижимость, автомобиль, образование, свадьба. С помощью функции, неправильные значения были заменены на верные и добавлены в новый столбец.

### Категоризация данных

In [18]:
# просматриваю уникальные значния столбца dob years сортируя по возрастанию
data['dob_years'].sort_values(ascending = True).unique()
# прописываю функцию для категоризации клиентов по возрасту
def age_group(age):
    if age < 35:
        return 'молодой возраст'
    elif age < 64:
        return 'средний возраст'
    else:
        return 'пенсионный возраст'
# добавляем новый столбец по категориям (применяя функцию)
data['age_group'] = data['dob_years'].apply(age_group)

# проверяю минимальный заработок
data['total_income'].min()
# проверяю максимальный заработок
data['total_income'].max()
# прописываю функцию для категоризации клиентов по уровню заработной платы
def income_group(income):
    if income < 35000:
        return 'низкий уровень дохода'
    elif income < 70000:
        return 'средний уровень дохода'
    elif income < 250000:
        return 'высокий уровень дохода'
    else:
        return 'очень высокий уровень дохода'
# добавляем новый столбец по категориям (применяя функцию)
data['income_group'] = data['total_income'].apply(income_group)



**Вывод** 

Произвела разбивку на категории в столбцах dob_years и total_income. Сделала функцию, которая возрващает категорию, после чего добавляю столбец и методом apply применяю фунцию к значениям в столбцах dob_years и total_income для записи категорий в новые столбцы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
# делаем сводную таблицу. Столбец, по которому группируем данные - children. Значения для сводной таблицы по колонке debt
# применяем функции sum, count, mean к значениям.
pivot_children = data.pivot_table(index = ['children'] , values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_children['mean'] = pivot_children['mean'] * 100
# именуем столбцы
pivot_children.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_children = pivot_children.sort_values(by = 'children', ascending = False)
# выводим
display(pivot_children)

,has_debt,total,%
children,,,
5,0,9,0.000000
4,4,41,9.756098
3,27,330,8.181818
2,202,2128,9.492481
1,445,4855,9.165808
0,1063,14091,7.543822


**Вывод**

 Глядя на таблицу можно сказать, что зависимость между наличием детей и возвратом кредита в срок имеется. Людей, у которых нет детей - процент по задолженности ниже на ≈2%, чем у людей, у которых есть дети. При этом люди, у которых 5 детей - задолженности не имеют вообще. Возможно, это связано с тем, что они никогда ранее не брали кредитов или с тем, что их слишком мало и не стоит учитывать их в общих выводах.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
# делаем сводную таблицу. Столбец, по которому группируем данные - family_status. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_family = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_family['mean'] = pivot_family['mean'] * 100
# именуем столбцы
pivot_family.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_family = pivot_family.sort_values(by = 'family_status', ascending = False)
# выводим
display(pivot_family)



,has_debt,total,%
family_status,,,
женат / замужем,931,12339,7.545182
гражданский брак,388,4151,9.347145
вдовец / вдова,63,959,6.569343
в разводе,85,1195,7.112971
Не женат / не замужем,274,2810,9.750890


**Вывод** 

Здесь мы так же можем сказать, что зависимость между семейным положением и возвратом кредита в срок есть. Наивысший процент по просрочкам у людей которые живут в гражданском браке и тех, кто не женат/ не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
# делаем сводную таблицу. Столбец, по которому группируем данные - income_group. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_income = data.pivot_table(index = ['income_group'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_income['mean'] = pivot_income['mean'] * 100
# именуем столбцы
pivot_income.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_income = pivot_income.sort_values(by = '%', ascending = False)
# выводим
display(pivot_income.head(15))


,has_debt,total,%
income_group,,,
высокий уровень дохода,1446,17168,8.422647
средний уровень дохода,98,1405,6.975089
очень высокий уровень дохода,194,2812,6.899004
низкий уровень дохода,3,69,4.347826


**Вывод**

По данном таблице можем сделать вывод, что зависимость между уровнем дохода и возвратом кредита в срок есть. Интересно, но у людей с низким уровнем дохода наблюдается наименьшее количество просрочек, что не скажешь про людей с высоким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:

# делаем сводную таблицу. Столбец, по которому группируем данные - lemmas_purposes. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_purposes = data.pivot_table(index = ['lemmas_purposes'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_purposes['mean'] = pivot_purposes['mean'] * 100
# именуем столбцы
pivot_purposes.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_purposes = pivot_purposes.sort_values(by = '%', ascending = False)
# выводим
display(pivot_purposes)


,has_debt,total,%
lemmas_purposes,,,
автомобиль,403,4306,9.359034
образование,370,4013,9.220035
свадьба,186,2324,8.003442
недвижимость,782,10811,7.233373


**Вывод**

По данным из таблицы можем сказать, что самая популярная цель кредита - это недвижимость, при этом процент по просрочкам у неё ниже всего, возможно потому, что недвижимость одна из самых ответственных целей (как мне кажется) - клиенты в первую очередь по возможности закрывают кредит, чтобы не оплачивать штрафы и пени. Самый высокий процент по просрочкам у клиентов, которые берут кредит на автомобиль и образование. Возможно, это связано с тем, что автомобиль и образование это не первостепенное в ежемесячном планировании затрат (что не скажешь о недвижимости), что влечет за собой бОльший процент по просрочкам. Менее популярная цель - свадьба, при этом процент по посрочкам у неё не самый высокий. Думаю объяснение тому идентично автомобилю и образованию.

## Шаг 4. Общий вывод

Исследование по надеждности заёмщиков показало, что количество детей и семейное положение влияет на факт погашения кредита в срок. В каждом из текущих вопросов имеется зависимость, о чем говорит разный процент по задолженности. При смене цели/количества детей/уровня дохода в каждой из категорий процент меняется, что говорит о зависмости.   